In [1]:
from cifar10_web import cifar10
import numpy as np

# Generating

In [2]:
X_train, y_train, X_test, y_test = cifar10(path=None)
X_train = X_train.reshape(50000,3,32,32).transpose(0,2,3,1)
#X_train /= X_train.max()
X_test = X_train.reshape(50000,3,32,32).transpose(0,2,3,1)
#X_test /= X_train.max()
y_train = np.array([np.argmax(a, axis=0) for a in y_train])
y_test = np.array([np.argmax(a, axis=0) for a in y_test])

# Class Selection

In [3]:
np.random.randint(low = 0, high = 10, size = 3)

array([4, 8, 6])

# index_selection

In [4]:
drop_ratio = [0.4, 0.75, 0.9]
X_train_selected = [X_train[y_train == label] for label in [5,7,4]]

In [5]:
def sample_index(labelled_data, drop_ratio):
    return np.random.choice(labelled_data.shape[0], int(np.ceil(labelled_data.shape[0]*(1 - drop_ratio))), replace=False)

In [6]:
indexes = [sample_index(class_, dr) for dr in drop_ratio for class_ in X_train_selected]

In [7]:
np.savez("selected_index.npz", indexes)

# Read

In [2]:
npzfile = np.load("selected_index.npz", allow_pickle = True)
indexes = npzfile["arr_0"]

In [3]:
def get_indexes(index_list, label: int = 5, drop_ratio: float = 0.4):
    assert(label in [5,7,4])
    assert(drop_ratio in [0.4, 0.75, 0.9])
    
    label_list = dict(zip([5,7,4],range(3)))
    drop_ratio_list = dict(zip([0.4, 0.75, 0.9],range(3)))
    return index_list[label_list[label]+3*drop_ratio_list[drop_ratio]]

In [4]:
get_indexes(indexes, label = 4, drop_ratio = 0.9)

array([1191, 2238, 4575, 1971, 4113, 2724,  194, 1256, 3204, 1618, 4816,
       3451, 3297, 1790, 1110, 1918, 2382, 3609, 3986, 1135, 4444, 4404,
       4215, 1490,  146, 3004, 4446, 1035, 2793, 1247, 1128, 3483,  150,
       3386, 4482, 3958, 1549, 2133, 4070, 3978,  214, 4032,  147, 4085,
       1678, 1258,  394, 2554, 4366, 4319, 4186, 3465, 1194, 3872, 1719,
       1186, 3618, 1267, 1957, 3804,  156, 2189, 2304, 3154, 4121,  794,
       3942, 4586, 1915, 4063, 2545, 2534, 4570, 3317,  776, 2163, 2540,
       1348, 1663, 4381, 2706, 2725, 4177,  885,  772, 3066, 2801, 3289,
       4164, 4241, 3644, 1450, 1465, 4493, 1641, 4507, 4696, 4536, 1778,
        878, 4693, 2641,  759, 2174, 2030, 2291, 3371, 4846, 4453, 2292,
       1355, 1369, 1706,  882, 2073, 4228, 2620, 1196, 4564, 1968, 4771,
       4567, 2096, 3721,  192, 2442, 3832, 4542,  274, 3069, 4491, 3130,
       1082, 1653, 4235, 3918, 1211, 1040,  654, 3515,  902, 2828, 4464,
        119, 2950, 2439,  500,  615,  190, 2003, 17